# 猜数3 (二分查找法)



1. 对于总数为1的猜数游戏, 最多1次一定能猜到结果
2. 对于总数为3的猜数游戏, 最多2次一定能猜到结果
   1. 先猜中间, 猜中了那就只要1次
   2. 没有猜中根据提示大小, 一定是左右之一
3. 对于总数为7的猜数游戏, 最多3次一定能猜到结果
   1. 先猜中间, 猜中了那就只要1次
   2. 没有猜中根据提示大小, 剩下的就是一个总数为3的猜数游戏, 根据2一定能够在2次内猜中
4. 对于总数为15的猜数游戏, 最多3次一定能猜到结果
   1. 先猜中间, 猜中了那就只要1次
   2. 没有猜中根据提示大小, 剩下的就是一个总数为7的猜数游戏, 根据3一定能够在3次内猜中
   
$a_n = a_{n-1} * 2 + 1$

$a_1 = 1$

求出: $a_n = 2^n - 1$

任意的已经排好序的数列, 总数为y, 如果$y+1 = 2^n$, 那么我们总能在 n次猜测内找到其中的一个随机数

$n = log_2 (y+1)$

那么如何用程序实现这种查找的算法呢?

首先我们重新定义一下`猜数`类

* 让它只在猜测正确的时候显示你猜了多少次, 防止其他的`print`刷屏
* 自动设置公式当中的最大猜测次数
* 可以指定这局游戏的答案(方便测试)

In [1]:
import random
import math

class 主持人():
    def __init__(self, 最小=1, 最大=10, 答案=0):
        if 答案:
            self.答案 = 答案
        else:
            self.答案 = random.randint(最小, 最大)
        self.猜过 = 0
        self.最大 = 最大
        self.最小 = 最小
        总数 = 最大 - 最小 + 1
        self.最多猜几次 = math.ceil(math.log2(总数 + 1))
    
    def 我猜(self, 数字):
        self.猜过 = self.猜过 + 1
        if self.猜过 > self.最多猜几次:
            return '你猜的次数太多了'
        if 数字 > self.答案:
            return '太大了'
        elif 数字 < self.答案:
            return '太小了'
        else:
            print(f'第{self.猜过}次猜测答对了')
            return '答对了'

某主持人 = 主持人()
print(f'答案:{某主持人.答案}, 最多猜{某主持人.最多猜几次}次')
for i in range(1, 11):
    print(某主持人.我猜(i))

答案:8, 最多猜4次
太小了
太小了
太小了
太小了
你猜的次数太多了
你猜的次数太多了
你猜的次数太多了
你猜的次数太多了
你猜的次数太多了
你猜的次数太多了


可以发现上面的暴力猜测方式会被游戏阻止, 那么如何正确地猜呢?

首先回想 

1. 总数是3的时候, 我们是先猜2, 再根据结果的大小, 猜测1/3
2. 总数是7的时候, 我们先猜测4, 然后剩下的部分就是123的猜数, 和567的猜数, 都用步骤一就可以了

总体的步骤就是先猜中间的, 然后根据大小继续猜测上/下面的一半, 重复上一步, 于是我们开始写实现

In [2]:
def 猜数策略(某主持人):
    最大 = 某主持人.最大
    最小 = 某主持人.最小
    while True:
        当前猜 = int((最大 + 最小)/2)
        结果 = 某主持人.我猜(当前猜)
        print(f'我猜: {当前猜}, 结果: {结果}')
        if 结果 == '太大了':
            最大 = 当前猜
        elif 结果 == '太小了':
            最小 = 当前猜
        elif 结果 == '答对了':
            return f'我赢了'
        else:
            return f'我错了...'


猜数策略(主持人())

我猜: 5, 结果: 太小了
第2次猜测答对了
我猜: 7, 结果: 答对了


'我赢了'

看起来像是那么回事, 但是如何确认我们的`猜数策略`函数写得是否正确呢?

这就要用到程序当中最经典的`暴力测试`

把答案是1~10的每一局游戏都测试一遍就好了

现实当中全部测试一遍比较困难, 但是对于程序来说并不成问题, 甚至只要几行代码而已, 这就是编程的魅力所在

In [3]:
def 测试(策略, 主持人类):
    for i in range(1, 11):
        print(f'==测试答案为{i}的猜数游戏==')
        print(策略(主持人类(答案=i)))
    
测试(猜数策略, 主持人)

==测试答案为1的猜数游戏==
我猜: 5, 结果: 太大了
我猜: 3, 结果: 太大了
我猜: 2, 结果: 太大了
第4次猜测答对了
我猜: 1, 结果: 答对了
我赢了
==测试答案为2的猜数游戏==
我猜: 5, 结果: 太大了
我猜: 3, 结果: 太大了
第3次猜测答对了
我猜: 2, 结果: 答对了
我赢了
==测试答案为3的猜数游戏==
我猜: 5, 结果: 太大了
第2次猜测答对了
我猜: 3, 结果: 答对了
我赢了
==测试答案为4的猜数游戏==
我猜: 5, 结果: 太大了
我猜: 3, 结果: 太小了
第3次猜测答对了
我猜: 4, 结果: 答对了
我赢了
==测试答案为5的猜数游戏==
第1次猜测答对了
我猜: 5, 结果: 答对了
我赢了
==测试答案为6的猜数游戏==
我猜: 5, 结果: 太小了
我猜: 7, 结果: 太大了
第3次猜测答对了
我猜: 6, 结果: 答对了
我赢了
==测试答案为7的猜数游戏==
我猜: 5, 结果: 太小了
第2次猜测答对了
我猜: 7, 结果: 答对了
我赢了
==测试答案为8的猜数游戏==
我猜: 5, 结果: 太小了
我猜: 7, 结果: 太小了
第3次猜测答对了
我猜: 8, 结果: 答对了
我赢了
==测试答案为9的猜数游戏==
我猜: 5, 结果: 太小了
我猜: 7, 结果: 太小了
我猜: 8, 结果: 太小了
第4次猜测答对了
我猜: 9, 结果: 答对了
我赢了
==测试答案为10的猜数游戏==
我猜: 5, 结果: 太小了
我猜: 7, 结果: 太小了
我猜: 8, 结果: 太小了
我猜: 9, 结果: 太小了
我猜: 9, 结果: 你猜的次数太多了
我错了...


测试结果发`猜数策略`函数对于答案是1~9的游戏都是正确的, 但是答案是10猜数游戏当中出错了

对于这种程序运行结果和我们预期不一样的现象, 我们称之为`bug`

那么出现`bug`了, 怎么解决呢?

先观察答案为10的游戏的猜测, 我们发现猜测9之后又猜测了9, 这明显不对, 于是我们修正下一次的猜测数字, 去除掉明显已经是错误的数字

In [4]:
def 完美猜数策略(某主持人):
    最大 = 某主持人.最大
    最小 = 某主持人.最小
    while True:
        当前猜 = int((最大 + 最小)/2)
        结果 = 某主持人.我猜(当前猜)
        print(f'我猜: {当前猜}, 结果: {结果}')
        if 结果 == '太大了':
            最大 = 当前猜 - 1
        elif 结果 == '太小了':
            最小 = 当前猜 + 1
        elif 结果 == '答对了':
            return f'我赢了'
        else:
            return f'我错了...'

测试(完美猜数策略, 主持人)

==测试答案为1的猜数游戏==
我猜: 5, 结果: 太大了
我猜: 2, 结果: 太大了
第3次猜测答对了
我猜: 1, 结果: 答对了
我赢了
==测试答案为2的猜数游戏==
我猜: 5, 结果: 太大了
第2次猜测答对了
我猜: 2, 结果: 答对了
我赢了
==测试答案为3的猜数游戏==
我猜: 5, 结果: 太大了
我猜: 2, 结果: 太小了
第3次猜测答对了
我猜: 3, 结果: 答对了
我赢了
==测试答案为4的猜数游戏==
我猜: 5, 结果: 太大了
我猜: 2, 结果: 太小了
我猜: 3, 结果: 太小了
第4次猜测答对了
我猜: 4, 结果: 答对了
我赢了
==测试答案为5的猜数游戏==
第1次猜测答对了
我猜: 5, 结果: 答对了
我赢了
==测试答案为6的猜数游戏==
我猜: 5, 结果: 太小了
我猜: 8, 结果: 太大了
第3次猜测答对了
我猜: 6, 结果: 答对了
我赢了
==测试答案为7的猜数游戏==
我猜: 5, 结果: 太小了
我猜: 8, 结果: 太大了
我猜: 6, 结果: 太小了
第4次猜测答对了
我猜: 7, 结果: 答对了
我赢了
==测试答案为8的猜数游戏==
我猜: 5, 结果: 太小了
第2次猜测答对了
我猜: 8, 结果: 答对了
我赢了
==测试答案为9的猜数游戏==
我猜: 5, 结果: 太小了
我猜: 8, 结果: 太小了
第3次猜测答对了
我猜: 9, 结果: 答对了
我赢了
==测试答案为10的猜数游戏==
我猜: 5, 结果: 太小了
我猜: 8, 结果: 太小了
我猜: 9, 结果: 太小了
第4次猜测答对了
我猜: 10, 结果: 答对了
我赢了


可以看到, 没有出现我错了的情况, 我们写了一个`测试`函数, 至少证明了在1~10之间的时候, 我们一定能在公式限定的次数内猜到正确答案

那么1~100呢?

### 练习1: 修改`测试`函数, 验证在猜数1~100, 也一定能猜到正确答案

可以看到我们只需要修改`测试`函数的一行就实现了这个功能, 甚至是只是添加一个0而已(什么? 你不是这么做的? 回去重写练习1)

但是输出太多了, 需要翻看很多页才能确保其中没有输出`我错了`(什么? 你知道`Ctrl+F`? 你很有灵性!)

如果下一个练习要求让你验证1~10000, 岂不是要累死. 第一节课程我们就讲到了DIY原则, 程序去替你做重复劳动

### 练习2: 设计一个`测试`函数, 验证猜数1~100, 给出最终的结论

其实我们不关心中间的输出, 猜测的细节,`测试`函数只要在`猜数策略`函数写得对的时候告诉我测试`通过`否则告诉告诉测试`失败`即可

完成下面`测试`函数, 让最后的输出结果 = (True, True)

In [5]:
def 测试(策略, 主持人类):
    # TODO 把实现写在这里
    pass

# 很显然, 我猜 函数的结论一定是失败, 因为1~10都猜错了, 何况1~100
# 而 我完美地猜 函数一定是通过的, 因为练习1 已经证明过了
测试(猜数策略, 主持人) == '失败', 测试(完美猜数策略, 主持人) == '通过'

(False, False)